### MechInterp

In [ ]:
step 1: F1
step 2: board 

In [ ]:
tokens = model.to_tokens(text)

In [ ]:
logits = model(tokens)

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
log_probs = F.log_softmax(logits[:, -1, :], dim=-1)

In [ ]:
target_tokens = tokens[1:]

In [ ]:
-log_probs.gather(dim=-1, index=target_tokens)

In [ ]:
step 1: ln1 = ln1 @ resid_pre
step 2: head_outputs = head1(ln1) + head2(ln2) ...
step 3: mid_resid = head_outputs + resid_pre
step 4: ln2 = ln2 @ mid_resid
step 5: mlp = mlp(ln2)
step 6: final_resid = mid_resid +mlp

In [ ]:
cache["hook_embed"]

In [ ]:
model.embed(tokens)

In [ ]:
step 1: diverse batch of data
step 2: record the attention pattenr of the target head
step 3: 

In [ ]:
W_out = model.W_out[2]

In [ ]:
_, cache = model.run_with_cache(tokens)

In [ ]:
w_in = cache["out", 2][:, 42]

In [2]:
from einops import einsum

In [ ]:
output = einsum(
    w_in, w_out
)

In [4]:
corrupted_prompts = [
    "When John and Mary went to the shops, Mary gave the bag to",
    "When Tom and James went to the park, Tom gave the ball to"
]

In [ ]:
correct_tokens = model.to_tokens("Mary Tom", prepend_bos=False)
incorrect_tokens = model.to_tokens("John James")

In [6]:
def compute_avg_logit_difference(logits, correct_tokens, incorrect_tokens):
    final_logits = logits[:, -1, :]
    correct_logits = final_logits[:, correct_tokens]
    incorrect_logits = final_logits[:, incorrect_tokens]
    return (correct_logits-incorrect_logits).mean()

In [ ]:
clean_tokens = model.to_tokens(clean_prompts)
corrupted_tokens = model.to_tokens(corrupted_prompts)

In [ ]:
clean_logits = model.run_with_hooks(clean_tokens)
corrupted_logits = model.run_with_hooks(corrupted_tokens)

In [ ]:
clean_logit_diff = compute_avg_logit_difference(clean_logits, correct_tokens, incorrect_tokens)
corrupted_logit_diff = compute_avg_logit_difference(corrupted_logits, correct_tokens, incorrect_tokens)

In [ ]:
def compute_ioi_metric(logits):
    logit_diff = compute_avg_logit_difference(logits, correct_tokens, incorrect_tokens)
    return (logit_diff-corrupted_logit_diff)/(clean_logit_diff-corrupted_logit_diff)

In [ ]:
cache, _ = model.run_with_cache(
    tokens,
    names_filter=lambda x: x.endswith("mlp_out")
)

In [ ]:
tokens = model.to_tokens(text)

In [ ]:
logits = model.run_with_hooks(tokens)

In [ ]:
log_probs = F.log_softmax(logits[:, -1, :], dim=-1)

In [ ]:
target_tokens = tokens[1:]

In [ ]:
predicted_log_probs = log_probs.gather(index=target_tokens, dim=-1)

step 1: choose a start component, and a receiver component
step 2: a clean prompt and a corrupted prompt
step 3: patch the activations fromt the clean corrupted prompt to the clean prompt, start_component
step 4: record the activations of the receiver component
step 5: rerun and patch the activations of the receiver

In [ ]:
_, cache = model.run_with_cache(past_moves)

In [ ]:
layer_idx = 3

In [ ]:
top_neurons = cache["post", layer_idx].std(dim=[0, 1]).argsort(descending=True)

In [7]:
heatmap_blanks = []

In [9]:
from einops import einsum

In [ ]:
def compute_similarity(neuron_idx, feature):
    w_out = model.W_out[layer_idx, neuron_idx, :]
    w_out /= w_out
    feature /= feature
    return einsum(
        w_out,
        feature,
        ""
    )

In [ ]:
for neuron_idx in top_neurons:
    heatmap_blanks.append(compute_similarity(
        neuron_idx, feature
    ))

In [ ]:
input_direction = model.W_in[layer_idx, :, neuron_idx]

In [ ]:
step 1: decompose all the inputs of the target head
step 2: calculate query and value component
step 3: query*value for each pair component

In [ ]:
embed = model.embed(tokens) + model.pos_embed(tokens)

In [ ]:
reisdual = embed

In [ ]:
for block in model.blocks:
    resid = block(resid)

In [ ]:
resid = model.ln_final(resid)
logits = model.unembed(resid)

In [ ]:
mine = linear_probe[..., 2]
theirs = linear_probe[..., 1]

In [ ]:
mine_vs_theirs = mine-theirs

In [ ]:
_, cache = model.run_with_cache(board_history)

In [10]:
n_layers = 6

In [11]:
from transformer_lens.utils import get_act_name

In [ ]:
def compute_contribution(output, feature):
    return einsum(
        output, feature,
    )

In [12]:
attn_contributions = []

In [ ]:
for layer_idx in range(n_layers):
    hook_name = get_act_name("attn_out", layer_idx)
    attn_out = cache[hook_name][2, 20, :]
    attn_contributions.append(compute_contribution(
        attn_out, mine_vs_theirsb
    ))

step 1: start with a diverse batch of data
step 2: record the activations of the target head
step 3: extract the attn pattern
step 4: average the attention pattern between the target query position and all other positions across batch
step 5: plot

In [ ]:
step 1: record activations

In [ ]:
the training task: training distribution, goal function
the base optimizer: architecture, sgd

In [ ]:
reachability
possible space
inductive biases

In [ ]:
register > cache > ram > disk > ex

In [13]:
class RegularStates:
    def __init__(self, states):
        self.states = states
    
    def restore(self):
        pass

In [ ]:
class State:
    def __init__(self, model, optim, **kwargs):
        kwargs.update({"model": model, "optim": optim})
        handlers, remainders = get_handlers(kwargs)
        
        for name, handler in handlers:
            setattr(self, name, handler)
        
        self.handlers = handlers
        
        super().__init__(remainders)
    
    def restore(self):
        for name, handler in self.handlers:
            handler.restore()
            
        super().restore()
    
    def commit(self):
        for name, handler in self.handlers:
            handler.commit()
        super().commit()

In [ ]:
node > pod > container

In [ ]:
class Pipeline:
    def __init__(self, batches, partitions, devices, scheduler=DetermisticScheduler()):
        self.batches = batches
        self.partitions = partitions
        self.devices = devices
        self.scheduler = scheduler
    
    def fit(self):
        n_batches = len(self.batches)
        n_partitions = len(self.partitions)
        
        with spawn_workers(self.devices) as (in_queues, out_queues):
            for schedule in self.scheduler.generate(n_batches, n_partitions):
                self._compute(schedule, in_queues, out_queue)
    
    def _compute(self, schedule, in_queues, out_queues):
        for microbatch_idx, parition_idx in schedule:
            batch = self.batches[microbatch_idx]
            partition = self.partitions[parition_idx]
            
            def compute_func():
                def wrapper():
                    return partition(batch)
            
            task = Task(compute_func)
            
            in_queues[microbatch_idx].put(task)
        
        for microbatch_idx, parition_idx in schedule:
            output = out_queues[microbatch_idx].get()
            in_queues[microbatch_idx] = output

In [ ]:
local_rank, local_world_size, ranks in a group, ProcessGroup

In [ ]:
step 1: global distributed group
step 2: parallel groups
step 3: set device
step 4: seed

In [ ]:
embedding, linear, attn, layer norm

In [14]:
import torch.distributed as dist

In [ ]:
for group in groups:
    dist.barrier()
    dist.destroy_process_group(group)

In [ ]:
step 1: get the input
step 2: construct a task
step 3: put the task in the correspond in_queues
step 4: wait and get the result
step 5: put the result in in_queue

In [ ]:
- storage
- minimize communication
- minimize flops

In [15]:
from copy import deepcopy

In [ ]:
class ModelStateHandler:
    def __init__(self, model):
        self.model = model
        
        self.saved_model_state = deepcopy(
            self.model.
        )
    
    def set_value(self, value):
        self.value = value
    
    def commit(self):
        self._saved_model_state = deepcopy(
            self.value.load_state_dict()
        )
    
    def restore(self):
        self.value.load_state_dict(self._saved_model_state)

In [16]:
def split_model(model, balances, devices):
    partitions = []
    partition_idx = 0
    layers = {}
    
    for name, module in model.named_children():
        layers[name] = module
        
        if len(layers) == balances[partition_idx]:
            partition = nn.Sequential(**layers)
            device = devices[partition_idx]
            partition.to(device)
            partitions.append(partition)
            partition_idx += 1
    
    return partitions

In [17]:
tensor_model_parallel_size = 2

In [18]:
pipeline_model_parallel_size = 4

In [19]:
num_pipeline_model_parallel_groups = 4

In [20]:
data_parallel_groups = []

In [21]:
for i in range(pipeline_model_parallel_size):
    start_rank = i*num_pipeline_model_parallel_groups
    end_rank = (i+1)*num_pipeline_model_parallel_groups
    
    for j in range(tensor_model_parallel_size):
        ranks = list(range(
            start_rank+j,
            end_rank,
            tensor_model_parallel_size
        ))
        
        print(ranks)

[0, 2]
[1, 3]
[4, 6]
[5, 7]
[8, 10]
[9, 11]
[12, 14]
[13, 15]


In [23]:
import torch.distributed as dist

In [ ]:
for group in groups:
    dist.barrier()
    dist.destroy_process_group(group)

In [ ]:
for param_group in optimizer.param_groups:
    for p in param_group["params"]:
        print(p.shape)

In [24]:
from torch.utils.data import DataLoader, random_split

In [ ]:
train_set, test_set = random_split(dataset, lengths=[6, 4])

In [ ]:
train_loader = DataLoader(train_set, batch_size=2)
test_loader = DataLoader(test_set, batch_size=2)

In [26]:
from torch.profiler import profile, ProfilerActivity

In [ ]:
with profile(
    activities=[ProfilerActivity.CPU]
)

In [28]:
from gymnasium import Env
from gymnasium.spaces import Discrete, Box

In [ ]:
class ShowerEnv(Env):
    def __init__(self):
        super().__init__()
        self.observation_space = Discrete()